In [2]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
comments = pd.read_csv('comments_en_cleaned.csv')
comments_en = comments[comments.lang == 'en']
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 10000,
                       min_df = 2, max_df = 0.95).fit(comments_en.Review)

In [4]:
pw = list(vect.vocabulary_.keys())

In [5]:
import enchant
c = enchant.Dict("en_UK")
def check_spelling(text):
    if not c.check(text):
        suggestions = list(set(c.suggest(text)).intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        else:
            res = text
    else:
        res = text
    return res

In [6]:
check_spelling('awsome')

'awesome'

In [7]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')
labeled3 = pd.read_excel('manual_labels3.xlsx')
labeled4 = pd.read_excel('manual_labels4.xlsx')
labeled5 = pd.read_excel('manual_labels5.xlsx')

In [8]:
labeled5.label.value_counts()

4    621
2    172
5     89
0     82
3     25
1     11
Name: label, dtype: int64

In [9]:
labeled2.shape

(1000, 5)

In [10]:
labeled1.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [11]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\!',' exclamation_point ', text)
    tokens = word_tokenize(deacc)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res

In [13]:
labeled = pd.concat([labeled1, labeled2, labeled3, labeled4, labeled5],axis = 0)

In [14]:
#legend = {
#    0: 'Other',
#    1: 'Crashes',
#    2: 'Balance/In game issues',
#    3: 'Synchronization/Loss',
#    4: 'Positive',
#    5: 'Bug'
#}

In [15]:
#labeled_output = labeled.loc[:, ['Review']]
#labeled_output['Label'] = labeled.label.map(legend)

In [17]:
#labeled_output.to_excel('manual_labels_output.xlsx')

In [18]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [19]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [57]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]
labeled_short = labeled[labeled.tokens.apply(len)<17]

In [58]:
labeled_short.shape

(3867, 7)

In [59]:
#labeled[labeled.label==1].to_excel('labeled_crash0.xlsx')
#labeled[labeled.label==2].to_excel('labeled_balance0.xlsx')
#labeled[labeled.label==3].to_excel('labeled_sync0.xlsx')
#labeled[labeled.label==4].to_excel('labeled_positive.xlsx')
#labeled[labeled.label==5].to_excel('labeled_bug0.xlsx')
#labeled[labeled.label==0].to_excel('labeled_other0.xlsx')

In [60]:
#comments_en['cleaned'] = comments_en.Review.apply(clean_comment)

In [61]:
#comments_en.to_csv('comments_en_cleaned.csv')

In [62]:
comments_en = pd.read_csv('comments_en_cleaned.csv')

In [63]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [64]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    
    class_names_b = ['neg', 'pos']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [65]:
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 10000,
                       min_df = 2, max_df = 0.95).fit(comments_en.cleaned)
vocab = vect.vocabulary_

In [69]:
from sklearn.pipeline import Pipeline
vect = CountVectorizer()
model = RidgeClassifier()
lin_model = Pipeline([('vectorizer', vect), ('classifier', model)])
lin_model.set_params(vectorizer__ngram_range = (1,3),vectorizer__analyzer = 'word', vectorizer__stop_words = 'english',
                     vectorizer__max_features = 800,
                     vectorizer__min_df = 2, vectorizer__max_df = 0.95,
                     vectorizer__vocabulary = vocab,
                     
                     classifier__class_weight = 'balanced', classifier__alpha = 0.25)
feats = labeled_short.cleaned
labels = labeled_short.label
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
lin_model = lin_model.fit(X_train, y_train)
y_train_pred = lin_model.predict(X_train)
y_test_pred = lin_model.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(lin_model, labeled.cleaned,labeled.label))))

train scores

                  precision    recall  f1-score   support

         unknown       0.60      0.68      0.64       266
           Crash       0.21      0.95      0.34        22
Balance problems       0.54      0.72      0.61       350
 Synchronization       0.27      1.00      0.42        30
        Positive       0.95      0.79      0.86      2250
             Bug       0.53      0.72      0.61       175

     avg / total       0.84      0.77      0.79      3093

test scores

                  precision    recall  f1-score   support

         unknown       0.54      0.59      0.56        59
           Crash       0.12      0.60      0.20         5
Balance problems       0.49      0.61      0.54       113
 Synchronization       0.15      0.45      0.22        11
        Positive       0.91      0.76      0.83       530
             Bug       0.45      0.52      0.48        56

     avg / total       0.77      0.70      0.73       774



/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


cross_val_score: 0.4914


In [68]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
vect = CountVectorizer()
model = RidgeClassifier()
lin_model = Pipeline([('vectorizer', vect), ('classifier', model)])
lin_model.set_params(vectorizer__analyzer = 'word', vectorizer__stop_words = 'english',
                     vectorizer__min_df = 2, vectorizer__max_df = 0.95,
                     
                     classifier__class_weight = 'balanced')

param_grid = {
        'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
        'vectorizer__max_features': [500,800,100,1200],        
        'classifier__alpha':[0.1, 0.2, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
    }

gs = GridSearchCV(lin_model, cv=3, n_jobs=-1, param_grid=param_grid, verbose = 1)
feats = labeled.cleaned
labels = labeled.label
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
gs = gs.fit(feats, labels)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which 

/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which 

/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which 

KeyboardInterrupt: 

In [59]:
gs.best_params_

{'classifier__C': 0.75,
 'vectorizer__max_features': 1200,
 'vectorizer__ngram_range': (1, 1)}

In [70]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return prediction[0]
def eval_pipeline(input_text, model = lin_model):
    cleaned_text = clean_comment(input_text)
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict([cleaned_text])
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_pipeline))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    print('model accuracy %1.4f'%(accuracy_score(y_true, y_pred)))
    return y_true,y_pred
y_true,y_pred = val_score(lin_model)

                  precision    recall  f1-score   support

           Other       1.00      0.90      0.95        10
           Crash       0.58      0.70      0.64        10
Balance problems       0.54      0.70      0.61        10
 Synchronization       0.83      0.50      0.62        10
        Positive       0.70      0.70      0.70        10
             Bug       0.60      0.60      0.60        10

     avg / total       0.71      0.68      0.69        60

model accuracy 0.6833


In [71]:
eval_pipeline('this dum gaem crashes every time i launch it')

1

In [72]:
eval_pipeline('tis a great game but those new weapons are so dam op')

2

In [73]:
eval_pipeline('The game glitched and all of my trophies and guns are now gone')

3

In [74]:
eval_pipeline('Cool!')

4

In [75]:
eval_pipeline('This game is haard to control')

5

In [46]:
val_en = pd.read_excel('validation_en.xlsx')
val_en_flattened = pd.concat([val_en.Other, val_en.Crash, val_en.Balance, val_en.Syncronization, val_en.Bug], axis = 0).reset_index()

In [47]:
val_en_flattened.head()

,index,0
0,0,Migs
1,1,Gamenya serusekali aku suka
2,2,No me gusto
3,3,En un perdida de tiempo yo queria secupera mi...
4,4,Happy2YearOld


In [48]:
val_en_flattened['dec'] = val_en_flattened[0].apply(eval_pipeline)
val_en_flattened.to_excel('linmodel_dec.xlsx')

In [66]:
labeled_crash = pd.read_excel('labeled_crash0.xlsx')

In [67]:
labeled_crash.subcat.value_counts()

2    25
1    14
0     7
Name: subcat, dtype: int64

In [44]:
from sklearn.pipeline import Pipeline
vect = CountVectorizer()
model = LogisticRegression()
lin_model = Pipeline([('vectorizer', vect), ('classifier', model)])
lin_model.set_params(vectorizer__ngram_range = (1,3),vectorizer__analyzer = 'word', vectorizer__stop_words = 'english',
                     vectorizer__max_features = 800,
                     vectorizer__min_df = 2, vectorizer__max_df = 0.95,
                     vectorizer__vocabulary = vocab,
                     
                     classifier__class_weight = 'balanced', classifier__C = 0.1)
feats = labeled_crash.cleaned
labels = labeled_crash.subcat
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
lin_model = lin_model.fit(X_train, y_train)
y_train_pred = lin_model.predict(X_train)
y_test_pred = lin_model.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(lin_model, labeled.cleaned,labeled.label))))

train scores

                  precision    recall  f1-score   support

         unknown       1.00      1.00      1.00         3
           Crash       1.00      0.75      0.86        12
Balance problems       0.88      1.00      0.93        21

     avg / total       0.93      0.92      0.91        36

test scores

                  precision    recall  f1-score   support

         unknown       0.00      0.00      0.00         4
           Crash       1.00      0.50      0.67         2
Balance problems       0.44      1.00      0.62         4

     avg / total       0.38      0.50      0.38        10



/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 6
  .format(len(labels), len(target_names))
/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


cross_val_score: 0.7876


In [46]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return prediction[0]
def eval_pipeline(input_text, model = lin_model):
    cleaned_text = clean_comment(input_text)
    class_names = ['Other',
        'Load',
        'In game']
    prediction = model.predict([cleaned_text])
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_pipeline))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    print('model accuracy %1.4f'%(accuracy_score(y_true, y_pred)))
    return y_true,y_pred
y_true,y_pred = val_score(lin_model)

                  precision    recall  f1-score   support

           Other       0.00      0.00      0.00        10
           Crash       0.67      0.20      0.31        10
Balance problems       0.18      1.00      0.30        10
 Synchronization       0.00      0.00      0.00        10
        Positive       0.00      0.00      0.00        10
             Bug       0.00      0.00      0.00        10

     avg / total       0.14      0.20      0.10        60

model accuracy 0.2000


/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
eval_pipeline()